In [1]:
import os

In [3]:
!pwd
os.chdir('../')

/d/PythonProjects/Text-summarizer/research


In [4]:
!Pwd

/d/PythonProjects/Text-summarizer


In [6]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: str 

In [7]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories, get_size

In [9]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])
        
    def get_data_transformation_config(self)-> DataTransformationConfig:
        
        config=self.config.data_transformation
        
        create_directories([config.root_dir])
        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name= config.tokenizer_name
        )
        return data_transformation_config        


In [10]:
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [13]:
class DataTransformation:
    def __init__(self, config:DataTransformationConfig):
        self.config=config
        self.tokenizer=AutoTokenizer.from_pretrained(self.config.tokenizer_name)
        
    def convert_to_features(self,batch):
        input_encodings=self.tokenizer(batch['dialogue'],padding=True, max_length=1024, truncation=True ,return_tensors='tf' )
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(batch['summary'],padding=True, max_length = 128, truncation = True, return_tensors='tf' )

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
        
    def convert_data(self):
        dataset_samsum=load_from_disk(self.config.data_path)
        dataset_samsum_pt=dataset_samsum.map(self.convert_to_features,batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [14]:
try:
    config=ConfigurationManager()
    data_transformation_config=config.get_data_transformation_config()
    data_transformation=DataTransformation(data_transformation_config)
    data_transformation.convert_data()
except Exception as e:
    raise e

[2023-08-02 21:43:37,640 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2023-08-02 21:43:37,643 : INFO : common : yaml file: params.yaml loaded successfully]
[2023-08-02 21:43:37,647 : INFO : common : created directory at: artifacts]
[2023-08-02 21:43:37,649 : INFO : common : created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

c:\Users\nagas\anaconda3\envs\gpu\lib\site-packages\transformers\tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14732 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/819 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/818 [00:00<?, ? examples/s]